import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
torch.cuda.is_available()

True

In [4]:
import pandas as pd

In [5]:
df= pd.read_excel("C:/Users/User/Desktop/논문준비/news/뉴스 차종 라벨링_스택_v2.xlsx",names=['date', 'sentence','source', 'label','pos_neg', 'contents'])

In [6]:
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

Number of training sentences: 4,999



In [7]:
# Display 10 random rows from the data.
df.sample(10)

,date,sentence,source,label,pos_neg,contents
3758,2일,"중부대, 학생학교지역사회 함께성장하는 'JB지역사랑드림프로젝트' 추진",경인일보,0,0,/중부대 제공 중부대는 오는 2021년 1학기부터 '학생~학교~지역사회'가 더불어 ...
3568,2일,"[동정] 장세용 구미시장, 사회적경제위원회 위원 위촉식 참석",뉴데일리,0,0,뉴데일리 강승탁 기자 = 장세용 구미시장은 14일 오후 3시 시청 국제통상협력실에서...
3035,22시간,"문 대통령 ""국민의 강화된 사회적 거리두기 동참으로 확산세 진정""",한국일보언론사 선정,0,0,"문재인 대통령이 13일 ""국민 여러분께서 강화된 사회적 거리두기에 한마음으로 동참해..."
2317,2020.08.01.,"[연예 마켓+] 뮤지컬도 되고, 야구도 되는데…왜 '미스터트롯'만 안되나요?",한국경제,0,0,"4차례 연기, 행정소송까지 이어졌던 '미스터트롯' 콘서트가 오는 8월 7일 개막 소..."
4448,2020.08.31.,"[기고] 노영희(서정여성문인회 회장 / 화성시 은빛독서나눔이) ""풀냄새""",경기도민일보,0,0,이른 아침이 좋아지기 시작한 것은 베란다에서 해가 뜨는 것을 보면서였다. 게으른 나...
3411,4일,"GSK, 사회공헌 활동도 언택트로",중앙일보 헬스미디어,0,0,코로나19 시대엔 사회공헌활동도 언택트로 진행한다. GSK는 임직원 봉사활동 참여 ...
220,2020.02.11.,현대차 울산공장 생산 재개..인기 차종부터,울산MBC,1,1,"다만 중국 현지에서 부품이 기대한 만큼 원활하게 공급되지는 않고 있어서, 인기 차종..."
1385,2019.06.30.,한국지엠 '新 SUV' 3총사 준비···내년 재도약 발판될까,전자신문,2,1,두 차종은 모두 미국에서 생산한 모델을 국내로 들여와 판매한다. 콜로라도는 국내 판...
3080,3일,"정부, 사회적 거리두기 2.5단계 재연장 '고심'…곳곳서 집단감염",한국경제언론사 선정,0,0,정부가 수도권의 사회적 거리두기 '2.5단계' 재연장 여부를 놓고 고심하고 있다. ...
2247,2020.08.27.,"서머너즈 워 글로벌 e스포츠 대회 ‘SWC 2020’, 3개월 대장정 시작",일간스포츠,0,0,e스포츠 대회 ‘서머너즈 워 월드 아레나 챔피언십 2020(SWC 2020)’이 막...


In [8]:
df.loc[df.label == 0].sample(5)[['sentence', 'label']]

,sentence,label
4480,"함평초등학교, 학습권 보장을 위한 독서확대기 지원",0
4119,'던전앤파이터X사이퍼즈' 공동 e스포츠 대회…언택트 진행,0
4357,"상주경찰서, 원룸 2곳 선정 범죄예방 우수시설 인증패 전달",0
4952,농산물은 다음 자산시장의 주인공 될까 [최준영의 경제 바로읽기],0
3302,"[포토]'사회적거리두기 2.5단계' 마지막날, 멈춰버린 도심",0


In [9]:
# Get the lists of sentences and their labels.
sentences = (df['sentence'].astype(str)+df['contents'].astype(str)).values
labels = df.pos_neg.values

In [10]:
sentences

array(["3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...",
       '힘 센 놈, 날렵한 놈, 신기한 놈…베일 벗는 차차차대표하는 차종들이 새 얼굴로 바뀌었다. 하반기에도 신차 여러 종이 출시를 준비 중이다. 8세대 신형... 3세대 K5, 쏘나타 질주 따라잡을까 최근 10여년 동안 거셌던 스포츠실용차(SUV) 바람에 가장 큰 영향을 받은 차종...',
       "S60·V60·V90 등 볼보 전 차종 미국서 리콜…'브레이크 결함'볼보가 미국에서 최근 판매된 전 차종에 대해 리콜을 실시한다. 31일 미국도로교통안전국(NHTSA) 등에 따르면 볼보는 현지에서 지난해부터 최근까지 판매된 S60·V60·S90·V90·XC40·XC60·XC90 등 모든 차량에 대한 리콜을...",
       ...,
       '<별별 구독경제>음식물 쓰레기 고온건조 분쇄… 세균 걱정 ‘끝’김철회 모두렌탈 이사는 “최근 소비 트렌드가 구독 경제로 변화하면서 가전제품 렌털도 증가하고 있는... 이어 “에코체의 경우 제휴카드 사용 등을 통해 하루 564원 상당의 경제적인 금액에 음식물 쓰레기...',
       '한은, 거시계량모형 BOK20 구축…"2012년 이후 경제상황 반영"한국은행은 2012년 이후 우리나라의 경제구조 변화를 반영하고 거시경제를 보다 현실적으로 분석하기 위해 기존의 \'거시계량모형\'을 보완한 BOK20 모형을 구축했다고 30일 밝혔다. 앞서 한은은 경제충격에 대한...',
       "파주시,'조선 최대의 실용백과사전 임원경제지' 학술대회 개최▲ 파주시,'조선 최대의 실용백과사전 임원경제지' 학술대회 개최 [경기 위클리오늘] 파주시는 오는 9월 3일 '조선 최대의 실용백과사전 임원경제지' 학술

In [11]:
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer

# Load the BERT tokenizer.
#print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

In [12]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...
Tokenized:  ['3', '##세', '##대', 'g', '##80', 'ᄎ', '##ᅮᆯ', '##시', '첫', '##날', '2', '##만', '##200', '##0', '##대', 'ᄀ', '##ᅨ', '##약', '[UNK]', '현', '·', 'ᄀ', '##ᅵ', 'ᄎ', '##ᅡ', '##종', "'", '역', '##대', 'ᄎ', '##ᅬ', '##다', "'", '이는', '현', '##대', '##차', '##와', 'ᄀ', '##ᅵ', '##아', '##차', ',', '제', '##네', '##시', '##스', '모든', 'ᄎ', '##ᅡ', '##종', '##중', '역', '##대', 'ᄎ', '##ᅬ', '##다', 'ᄀ', '##ᅨ', '##약', '##건', '##수', '##다', '.', '또한', 'ᄋ', '##ᅩᆯ', 'ᄑ', '##ᅡᆫ', '##매', '##량', '3', '##만', '##30', '##00', '##대의', '3', '##분', '##의', '2', '##가', '첫', '날', '##에', 'ᄑ', '##ᅡᆯ', '##린', '세', '##ᆷ이', '##다', '.', '제', '##네', '##시', '##스는', '30일', '온', '##라', '##인', '##으로', 'ᄎ', '##ᅮᆯ', '##시', '##행', '##사를', '가', '##ᆽ고', 'g', '##80', '##의'

In [13]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  177


In [14]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\User\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please 

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Original:  3세대 G80 출시 첫날 2만2000대 계약…현·기 차종 '역대 최다'이는 현대차와 기아차, 제네시스 모든 차종중 역대 최다 계약건수다. 또한 올 판매량 3만3000대의 3분의 2가 첫 날에 팔린 셈이다. 제네시스는 30일 온라인으로 출시행사를 갖고 G80의 완전 변경 모델 디 올 뉴 G80를...
Token IDs: tensor([  101,   124, 25169, 14266,   149, 41282,  1177, 65633, 14280, 26037,
        41581,   123, 18823, 82595, 10995, 14266,  1163, 61701, 46190,   100,
        42703,   193,  1163, 32035,  1177, 25539, 22516,   112, 13183, 14266,
         1177, 31496, 12261,   112, 33802, 42703, 14266, 23724, 12747,  1163,
        32035, 18253, 23724,   117, 59354, 72197, 14280, 13212, 26019,  1177,
        25539, 22516, 43312, 13183, 14266,  1177, 31496, 12261,  1163, 61701,
        46190, 66921, 15783, 12261,   119, 20124,  1174, 73057,  1180, 26646,
        94083, 43620,   124, 18823, 30086, 21334, 47615,   124, 31049, 10576,
          123, 11376, 26037, 76818, 10609,  1180, 82720, 35018, 47257, 41808,
        12261,   119, 59354, 72197, 14280, 60605, 40415, 82582, 18073, 12680,
        11945,  1177, 65633, 1

In [15]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,499 training samples
  500 validation samples


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(df['label'].unique()), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [18]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (105879, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [19]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [20]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [21]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [23]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:13.
  Batch    80  of    282.    Elapsed: 0:00:25.
  Batch   120  of    282.    Elapsed: 0:00:38.
  Batch   160  of    282.    Elapsed: 0:00:52.
  Batch   200  of    282.    Elapsed: 0:01:05.
  Batch   240  of    282.    Elapsed: 0:01:18.
  Batch   280  of    282.    Elapsed: 0:01:33.

  Average training loss: 0.43
  Training epcoh took: 0:01:33

Running Validation...
  Accuracy: 0.908
  Validation Loss: 0.25
  Validation took: 0:00:03

======== Epoch 2 / 2 ========
Training...
  Batch    40  of    282.    Elapsed: 0:00:16.
  Batch    80  of    282.    Elapsed: 0:00:32.
  Batch   120  of    282.    Elapsed: 0:00:47.
  Batch   160  of    282.    Elapsed: 0:01:03.
  Batch   200  of    282.    Elapsed: 0:01:18.
  Batch   240  of    282.    Elapsed: 0:01:34.
  Batch   280  of    282.    Elapsed: 0:01:50.

  Average training loss: 0.18
  Training epcoh took: 0:01:51

Running Validation...
  Accuracy: 0.936

In [24]:
torch.cuda.empty_cache()